<a href="https://colab.research.google.com/github/mohmiim/MLIntroduction/blob/master/session-4/Session_4_Data_Augmentationt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf

if tf.__version__.startswith('2')==False :
  !pip uninstall tensorflow
  !pip install tensorflow-gpu
print(tf.__version__)


2.1.0


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.optimizers import Adam




In [0]:

# https://github.com/mohmiim/MLIntroduction/raw/master/data/data.zip
# https://github.com/mohmiim/MLIntroduction/raw/master/data/smallData.zip



In [8]:
import os

!wget --no-check-certificate \
    https://github.com/mohmiim/MLIntroduction/raw/master/data/data.zip \
    -O /tmp/data.zip

--2020-03-26 22:06:52--  https://github.com/mohmiim/MLIntroduction/raw/master/data/data.zip
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mohmiim/MLIntroduction/master/data/data.zip [following]
--2020-03-26 22:06:52--  https://raw.githubusercontent.com/mohmiim/MLIntroduction/master/data/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19729033 (19M) [application/zip]
Saving to: ‘/tmp/data.zip’

/tmp/data.zip       100%[===================>]  18.81M  --.-KB/s    in 0.1s    

2020-03-26 22:06:53 (159 MB/s) - ‘/tmp/data.zip’ saved [19729033/19729033]



In [0]:
import zipfile
def unzip(file,target):
  zip_ref = zipfile.ZipFile(file, 'r')
  zip_ref.extractall(target)
  zip_ref.close()

unzip("/tmp/data.zip","/tmp")

In [10]:

print(os.listdir("/tmp/data"))


['.gitignore', 'train', 'test']


In [0]:
#default sizes
Image_Width = 100
Image_Height = 100
Image_Depth = 3
targetSize = (Image_Width,Image_Height)
targetSize_withdepth = (Image_Width,Image_Height,Image_Depth)
epochs = 200

In [13]:
#define the sub folders for both training and test
training = os.path.join("/tmp/data",'train')

#now the easiest way to load data is to use the ImageDataGenerator
train_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          zoom_range=0.2,
                                          fill_mode='nearest')


train_generator = train_data_generator.flow_from_directory(training,
                                                           batch_size=20,
                                                           target_size=targetSize,
                                                           #seed=12
                                                           )

Found 229 images belonging to 5 classes.


In [14]:
model = Sequential()
model.add(Conv2D(128,(3,3),input_shape=targetSize_withdepth,activation=relu))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64,(3,3),activation=relu))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(32,(3,3),activation=relu))
model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(Dense(512,activation=relu))
model.add(Dense(5,activation='softmax'))
history = model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 128)       3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0

In [0]:
model.fit_generator(generator=train_generator,epochs=epochs)


In [0]:
from sklearn.metrics import confusion_matrix, classification_report
def test(generator, model):
  predictions = model.predict_generator(generator)
  row_index = predictions.argmax(axis=1)
  target_names = generator.class_indices.keys()
  print(target_names)
  print(confusion_matrix(generator.classes, row_index))
  print('Classification Report')
  print(classification_report(generator.classes, row_index, target_names=target_names))

In [19]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("/tmp/data/train",
                                                         target_size=(100,100),
                                                         shuffle=False)
test(generator=test_generator, model=model)

Found 229 images belonging to 5 classes.
Instructions for updating:
Please use Model.predict, which supports generators.


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


dict_keys(['bar_chart', 'bubble_chart', 'pie_chart', 'radar_chart', 'treemap_chart'])
[[39  0  0  0  0]
 [ 0 49  0  1  0]
 [ 0  0 56  0  0]
 [ 0  0  0 51  0]
 [ 0  0  0  0 33]]
Classification Report
               precision    recall  f1-score   support

    bar_chart       1.00      1.00      1.00        39
 bubble_chart       1.00      0.98      0.99        50
    pie_chart       1.00      1.00      1.00        56
  radar_chart       0.98      1.00      0.99        51
treemap_chart       1.00      1.00      1.00        33

     accuracy                           1.00       229
    macro avg       1.00      1.00      1.00       229
 weighted avg       1.00      1.00      1.00       229



In [20]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("/tmp/data/test",
                                                         target_size=(100,100),
                                                         shuffle=False)

Found 93 images belonging to 5 classes.


In [21]:
test(generator=test_generator, model=model)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


dict_keys(['bar_chart', 'bubble_chart', 'pie_chart', 'radar_chart', 'treemap_chart'])
[[17  0  1  0  1]
 [ 0 12  0  7  0]
 [ 0  2 12  2  2]
 [ 0  3  1 14  0]
 [ 0  1  2  0 16]]
Classification Report
               precision    recall  f1-score   support

    bar_chart       1.00      0.89      0.94        19
 bubble_chart       0.67      0.63      0.65        19
    pie_chart       0.75      0.67      0.71        18
  radar_chart       0.61      0.78      0.68        18
treemap_chart       0.84      0.84      0.84        19

     accuracy                           0.76        93
    macro avg       0.77      0.76      0.76        93
 weighted avg       0.78      0.76      0.77        93

